In [ ]:
import time
import logging
from logging.config import dictConfig
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, morphology, exposure, transform, color
from skimage.measure import label, regionprops
from random import randint
import sys
import logging
from logging.config import dictConfig

def main():
    filenames = ["test_im1.bmp","test_im2.bmp"]
    for file in filenames:
        for windowSize in [5,9,11]:
            start = time.time()
            textureSynthesis("img/"+file, windowSize)
            end = time.time()
            logging.info("\t"+file+"-"+str(windowSize)+"\t:-  "+str(end-start)+" secs")

if __name__ == '__main__':
    main()

In [ ]:
def textureSynthesis(imageFile, windowSize):

    print(imageFile.split('.')[0]+"-"+str(windowSize)+"-synth.gif")
    img = io.imread(imageFile)
    img = img/255.0
    row, col = np.shape(img)

    error_threshold = 0.1
    Maxerror_threshold = 0.3
    sigma = windowSize/6.4
    seed = 3
    halfWindow = (windowSize - 1) // 2
    totalPixels = img.shape[0] * img.shape[1]
    print("---------")
    print(img)
    print("---------")
    filledMap = np.ceil(img)
    filledPixels = np.sum(filledMap)

    convPatches = convolutionPatches_mod(img, filledMap, halfWindow)
    print(convPatches.shape)
    synthesizedImage = img

    gaussMask = GaussMask(windowSize,sigma)

    synthImagePad = np.lib.pad(synthesizedImage, halfWindow, mode='constant', constant_values=0)
    filledMapPad = np.lib.pad(filledMap, halfWindow, mode='constant', constant_values=0)

    print(filledPixels)
    print(totalPixels)

    while filledPixels < totalPixels:
        progress = False
        pixelList = np.nonzero(morphology.binary_dilation(filledMap) - filledMap)
        neighbors = []
        neighbors.append([np.sum(filledMap[pixelList[0][i] - halfWindow : pixelList[0][i] + halfWindow + 1, pixelList[1][i] - halfWindow : pixelList[1][i] + halfWindow + 1]) for i in range(len(pixelList[0]))])
        decreasingOrder = np.argsort(-np.array(neighbors, dtype=int))

        for i in decreasingOrder[0]:
            template = synthImagePad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            validMask = filledMapPad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            bestMatches = findMatches(template,convPatches,validMask,gaussMask,windowSize, halfWindow, error_threshold)

            bestMatch = randint(0, len(bestMatches)-1)

            if bestMatches[bestMatch][0]<=Maxerror_threshold:
                synthImagePad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = bestMatches[bestMatch][1]
                synthesizedImage[pixelList[0][i]][pixelList[1][i]] = bestMatches[bestMatch][1]
                filledMapPad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = 1
                filledMap[pixelList[0][i]][pixelList[1][i]] = 1
                filledPixels+=1

                progress = True
        if not progress:
            Maxerror_threshold *= 1.1
        i = (filledPixels/totalPixels)*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush()
    io.imsave(imageFile.split('.')[0]+"-"+str(windowSize)+"-synth.gif", synthesizedImage)
    plt.show()
    return


def GaussMask(windowSize, sigma):
    x, y = np.mgrid[-windowSize//2 + 1:windowSize//2 + 1, -windowSize//2 + 1:windowSize//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()


def convolutionPatches_mod(img, filledMap, halfWindow):
    convPatches = []
    c = 0
    for i in range(halfWindow, img.shape[0] - halfWindow - 1):
        for j in range(halfWindow, img.shape[1] - halfWindow - 1):
            if 0 in filledMap[i - halfWindow:i + halfWindow + 1, j - halfWindow: j + halfWindow + 1]:
                c = c + 1
            else:
                convPatches.append(np.reshape(
                    img[i - halfWindow:i + halfWindow + 1, j - halfWindow: j + halfWindow + 1],
                    (2 * halfWindow + 1) ** 2))
    convPatches = np.double(convPatches)
    return convPatches

In [ ]:
def findMatches(template,convPatches, valid_mask, gauss_mask, windowSize, halfWindow, error_threshold):

    template = np.reshape(template, windowSize*windowSize)
    gauss_mask = np.reshape(gauss_mask, windowSize*windowSize)
    valid_mask = np.reshape(valid_mask, windowSize*windowSize)
    total_weight = np.sum(np.multiply(gauss_mask, valid_mask))
    distance = (convPatches-template)**2
    ssd = np.sum((distance*gauss_mask*valid_mask) / total_weight, axis=1)
    min_error = min(ssd)
    j = int(((2 * halfWindow + 1) ** 2) / 2)
    return [[err, convPatches[i][j]] for i, err in enumerate(ssd) if err <= min_error*(1+error_threshold)]